### Description

In [2]:
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #
# **************************************    FAKER DATA SCRIPTS                     ******************************************************* #
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #
# Author: Miguel Mares
# Date: 12-16-2024
# Description: Faker data notebook to generate fake blog posts, which are then imported and made public through website. 
# **************************************************************************************************************************************** #

# library/package imports


# List of authors from DB
# **************************
# Drazenka Jelic
# Ida Celma
# Hoc Tran
# David safranek
# Svetlana Todorovic
# Richard Zahradnicek
# Seo-yun Paik
# Emily Whittle



In [78]:
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #
# IMPORTS
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #

from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from faker.providers import BaseProvider
from faker import Faker
import subprocess
import pyodbc
import random



# Replace with your Key Vault URL
key_vault_url = "https://mmgwkv.vault.azure.net/"

# Create a DefaultAzureCredential instance
credential = DefaultAzureCredential()

# Create a SecretClient instance
client = SecretClient(vault_url=key_vault_url, credential=credential)

# Retrieve the username, password and dbName
username_secret = client.get_secret("mmgwsqluser")
password_secret = client.get_secret("mmgwsqlpwd")
server_name = client.get_secret("mmgwsqlserver")
db_name = client.get_secret("mmgwsqldbname")

username = username_secret.value
password = password_secret.value
server = server_name.value
db = db_name.value

In [80]:
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #
# GENERAL FUNCTIONS AND CLASSES
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #

# azd login function to not have to run a powershell script.
def azd_auth_login():
    try:
        # Run the azd auth login command
        result = subprocess.run(['azd', 'auth', 'login'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Error: {e.stderr.decode()}")


# Custom provider to create custom tailored content. 
class InternalBlogProvider(BaseProvider):
    def InternalTechPosts(self):
        subjects = [
            "Internal Artificial Intelligence Research",
            "Machine Learning Project",
            "Data Science Research",
            "Cloud Computing Research",
            "Quantum Computing Research"
        ]

        authors = [
                {"author":"Kayla Woodcock","PersonID":2},
                {"author":"Hudson Onslow","PersonID":3},
                {"author":"Isabella Rupp","PersonID":4},
                {"author":"Eva Muirden","PersonID":5},
                {"author":"Sophia Hinton","PersonID":6},
                {"author":"Amy Trefl","PersonID":7},
                {"author":"Anthony Grosse","PersonID":8},
                {"author":"Alica Fatnowna","PersonID":9},
                {"author":"Stella Rosenhain","PersonID":10},
                {"author":"Ethan Onslow","PersonID":11},
                {"author":"Henry Forlonge","PersonID":12},
                {"author":"Hudson Hollinworth","PersonID":13},
                {"author":"Lily Code","PersonID":14},
                {"author":"Taj Shand","PersonID":15},
                {"author":"Archer Lamble","PersonID":16},
                {"author":"Piper Koch","PersonID":17},
                {"author":"Katie Darwin","PersonID":18},
                {"author":"Jai Shand","PersonID":19},
                {"author":"Jack Potter","PersonID":20}
        ]
        
        subject = self.random_element(subjects)
        author = self.random_element(authors)

        return {
            "title": f"{self.generator.catch_phrase()}",
            "author": author["author"],
            "publish_date": self.generator.date_time_this_year().isoformat(),
            "content": "\n".join(self.generator.paragraphs(nb=5)),
            "tags": ", ".join([subject.lower(), "technology", "innovation", "internal"]),
            "personID": author["PersonID"]
        }


# Connect to SQL Server
def connectToSql():
    try:
        conn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};'
                            f'SERVER={server};'
                            f'DATABASE={db};'
                            f'UID={username};'
                            f'PWD={password}')
        return conn
    except pyodbc.OperationalError as e:
        print(f"OperationalError: {e}")


In [82]:
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #
# CRUD OPERATIONS
# **************************************************************************************************************************************** #
# **************************************************************************************************************************************** #
# 

# Call the function to authenticate and login
azd_auth_login()

# Creating Cursor for insert
conn = connectToSql()
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'Application' AND TABLE_NAME = 'BlogPosts')
CREATE TABLE Application.BlogPosts (
    id INT IDENTITY(1,1) PRIMARY KEY,
    title NVARCHAR(255),
    author NVARCHAR(255),
    publish_date DATETIME,
    content TEXT,
    tags NVARCHAR(255),
    personID INT,
    FOREIGN KEY (personID) REFERENCES Application.People(PersonID)
)
''')


# Add the custom provider to Faker
fake = Faker()
fake.add_provider(InternalBlogProvider)

# Generate a list of fake tech blog posts
internal_blog_posts = [fake.InternalTechPosts() for _ in range(1000)]

# Insert the blog posts into the database
for post in internal_blog_posts:
    cursor.execute('''
    INSERT INTO Application.BlogPosts (title, author, publish_date, content, tags, personID)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (post['title'], post['author'], post['publish_date'], post['content'], post['tags'], post['personID']))



# Commit the transaction
conn.commit()

# Close the connection
cursor.close()
conn.close()



